<a href="https://colab.research.google.com/github/sruthi2498/PanaromaAnalysis/blob/main/MotionTrails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import collections
import os
import tempfile
import copy

from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import urllib

import tensorflow as tf

import cv2 as cv
from PIL import Image
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
fg_frames=[]
folder = "test1/foreground2/"
image_num = len(os.listdir(folder))
for i in range(1,image_num+1):
    fg_frames.append(cv.imread(folder+str(i)+'.png'))

len(fg_frames)

In [ ]:
pano = cv.imread("panorama.png")

In [ ]:
sift = cv.xfeatures2d.SIFT_create()

def match(img1,img2):
  kp1, des1 = sift.detectAndCompute(img1, None)
  kp2, des2 = sift.detectAndCompute(img2, None)
      # Apply ratio test
  bf = cv.BFMatcher()
  matches = bf.knnMatch(des1, des2, k=2)
      # Apply ratio test
  good_matches = []
  good_matches_2 = []
  for m, n in matches:
    if m.distance < 0.5 * n.distance:
        good_matches.append([m])
        good_matches_2.append(m)

  if len(good_matches)>50:
    good_matches_2 = sorted(good_matches_2, key = lambda x:x.distance)
    sift_matches = cv.drawMatchesKnn(img1, kp1, img2, kp2, good_matches, outImg=None, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    # plt.figure(figsize=(20, 10), dpi=80)

    # plt.imshow(sift_matches),plt.show()
    return good_matches_2,kp1,kp2
  return None,None,None

In [ ]:
def getQueryAndTrainPts(good_matches,kp1,kp2):
  src_pts = np.float32([ kp1[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
  dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2) 
  return src_pts,dst_p


def makewhiteBlack(frame):
  frame2 = frame.copy()
  white = np.all(frame == [255, 255, 255], axis=-1)
  frame2[white] =0
  return frame2


In [ ]:
factor = 1/4
img2 = pano.copy()
H,W,_ = img2.shape
for i in range(len(frames)):
  img1_og = frames[i]
  fg_img = rescale(fg_frames[i],factor)

  img1 = rescale(img1_og,factor)
  good_matches,kp1,kp2 = match(img1,img2)
  if good_matches:
    print(i)
    img2_h,img2_w = img2.shape[:-1]
    src_pts,dst_pts = getQueryAndTrainPts(good_matches,kp1,kp2)

    M, mask = cv.findHomography( src_pts,dst_pts,cv.RANSAC,5.0)
    M = M*1/factor

    fg_img = makewhiteBlack(fg_img)
    fg_img_out = cv.warpPerspective(fg_img.copy(), M, (img2.shape[1], img2.shape[0] ))
    rows,cols,channels = fg_img_out.shape
    roi = img2[0:H,0:W]
    img2gray = cv.cvtColor(fg_img_out,cv.COLOR_BGR2GRAY)
    ret, mask = cv.threshold(img2gray, 1, 255, cv.THRESH_BINARY)
    mask = cv.dilate(mask, None, iterations=3)
    mask = cv.erode(mask, None, iterations=5)
    mask_inv = cv.bitwise_not(mask)
    img2_bg = cv.bitwise_and(roi,roi,mask = mask_inv)
    img1_fg = cv.bitwise_and(fg_img_out,fg_img_out,mask = mask)
    dst = cv.add(img2_bg,img1_fg)
    img2[0:H,0:W] = dst

    cv2_imshow(img2)



In [ ]:
cv.imwrite("motion_trails.png",img2)